In [1]:
import os
from vedo import *
import vtk
from vmtk import pypes
import traceback
import pickle
import networkx as nx
from auxiliares import read_vtk, save_r_p, read_obj
from parseObj import calcularMatriz, calcularMatrizSplines
import Arbol as modelo
from ResamplingRDP import interpolarRDP_conRadio, vtpToObj
from splines import limpiarRadiosSplines, binarizar, traversefeaturesSerializado, calculate_splines, binarizar, grafo2arbol
from collections import Counter
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
from scipy.interpolate import splprep, splev, make_splprep, BSpline
import matplotlib.pyplot as plt

Obtain centerline

In [2]:
files = os.listdir('../modelsNormalizado/vesselsNormalized')

FileNotFoundError: [Errno 2] No such file or directory: '../modelsNormalizado/vesselsNormalized'

In [3]:
#files = os.listdir('../models/vessels/remeshed/area-005')

cfolder = os.listdir('../modelsNormalizado/centerlines')
for file in files:
    if file.split(".")[1] == "vtp":
        if file.split(".")[0] + '-network.vtp' not in cfolder:
            print("Processing file:", file)
            script = 'vmtknetworkextraction '
            input_file =  '-ifile ../modelsNormalizado/vesselsNormalized/' + file 
            output_file = ' -ofile ../modelsNormalizado/centerlines/' + file.split(".")[0] + '-network.vtp'
            myPype = pypes.PypeRun(script+input_file+output_file)

FileNotFoundError: [Errno 2] No such file or directory: '../modelsNormalizado/centerlines'

In [4]:
centerlines = os.listdir("../modelsNormalizado/centerlines/")
reader = vtk.vtkXMLPolyDataReader()
writer = vtk.vtkXMLPolyDataWriter()
for file in centerlines:
    if file not in os.listdir("../modelsNormalizado/centerlinesResampled/") and file.split(".")[1] == "vtp":
        print("calculando: ", file)
        reader.SetFileName("../modelsNormalizado/centerlines/" + file)
        reader.Update()
        # Get the polydata
        centerline = reader.GetOutput()
        print("Centerline points:", centerline)
        resampleada = interpolarRDP_conRadio(centerline, 0.02)
        writer.SetFileName("../modelsNormalizado/centerlinesResampled/" + file)
        writer.SetInputData(resampleada)
        writer.Write()

centerline to obj

In [5]:
centerfolder = os.listdir("../modelsNormalizado/centerlinesResampled/")
objfolder = os.listdir("../modelsNormalizado/centerlinesResampledOBJ/")
#arrays_to_remove = ['EdgeArray', 'EdgePCoordArray', 'TCoords_']
for file in centerfolder:
    if file.split(".")[1] == "vtp" and (file.split(".")[0] + '.obj') not in objfolder:
        print("converting", file)
        centerline = read_vtk("../centerlines/" + file)            
        vtpToObj(file, "../modelsNormalizado/centerlinesResampled","../modelsNormalizado/centerlinesResampledOBJ")

Mesh vtp to obj

In [6]:
objMfolder = os.listdir("../modelsNormalizado/vesselsOBJ")
files = os.listdir("../modelsNormalizado/vesselsNormalized")
#arrays_to_remove = ['EdgeArray', 'EdgePCoordArray', 'TCoords_']
for file in files:
    if file.split(".")[1] == "vtp" and (file.split(".")[0] + '.obj') not in objMfolder:
        print("converting")
        vtpToObj(file, "../modelsNormalizado/vesselsNormalized","../modelsNormalizado/vesselsOBJ")

create radius array

In [7]:
# Input and output directories
input_dir = "../modelsNormalizado/centerlinesResampled"
output_dir = "../modelsNormalizado/radius_arrays"
os.makedirs(output_dir, exist_ok=True)

# VTP reader
reader = vtk.vtkXMLPolyDataReader()

# Iterate through all .vtp files in the folder
for filename in os.listdir(input_dir):
    if filename.endswith(".vtp"):
        filepath = os.path.join(input_dir, filename)

        # Read the VTP file
        reader.SetFileName(filepath)
        reader.Update()
        polydata = reader.GetOutput()

        # Get the Radius array from PointData
        point_data = polydata.GetPointData()
        radius_array = point_data.GetArray("Radius")

        if radius_array is not None:
            radius_np = vtk_to_numpy(radius_array)

            # Save to .npy file
            output_filename = os.path.splitext(filename)[0] + "_radius.npy"
            output_path = os.path.join(output_dir, output_filename)
            np.save(output_path, radius_np)
            print(f"Saved: {output_path}")
        else:
            print(f"No 'Radius' array found in {filename}")

Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_09-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_16-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_17_10-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_27-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_28_269-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_30_612-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_30_614-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_33_628-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_33_631-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_34-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\ANSYS_UNIGE_35-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\C0001-network_radius.npy
Saved: ../modelsNormalizado/radius_arrays\C0002-network_radius.npy
Saved: 

create splines

In [8]:
centerfolder = "../modelsNormalizado/centerlinesResampled/"
meshfolder = "../modelsNormalizado/vesselsNormalized/"
coef_folder = "../modelsNormalizado/splines/coeficientes/"


centerlines = os.listdir(centerfolder)
centerlines = sorted(centerlines)  
centerlines = centerlines
meshes = os.listdir(meshfolder)

meshes = sorted(meshes)
meshes = meshes 

for mesh in meshes:
    calculate_splines(mesh, coef_folder, centerfolder, meshfolder)

create graph radio

In [9]:
gfolder = os.listdir('../modelsNormalizado/grafos')
l_error = []
meshOBJ = os.listdir("../modelsNormalizado/vesselsOBJ")

for file in meshOBJ:
    print(file)
    if file.split(".")[1] is not None:
        if file.split(".")[1] == "obj":
            fileObj = open("../modelsNormalizado/centerlinesResampledOBJ/" +file.split(".")[0]+"-network.obj")
            if file.split(".")[0] + '-grafo.gpickle' not in gfolder:
                try: 
                    grafo = calcularMatriz(fileObj, "../modelsNormalizado/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
                    print("calculating: ", file)
                    with open("../modelsNormalizado/grafos/" + file.split(".")[0] + '-grafo.gpickle', 'wb') as f:
                        pickle.dump(grafo, f, pickle.HIGHEST_PROTOCOL)
                except Exception:
                    print("problem with: ", file)
                    l_error.append(file)
                    traceback.print_exc()
gfolder = os.listdir('../modelsNormalizado/grafos')   
#print("problem with files: ", l_error)

ANSYS_UNIGE_09.mtl
ANSYS_UNIGE_09.obj
problem with:  ANSYS_UNIGE_09.obj
ANSYS_UNIGE_16.mtl
ANSYS_UNIGE_16.obj
problem with:  ANSYS_UNIGE_16.obj
ANSYS_UNIGE_17_10.mtl
ANSYS_UNIGE_17_10.obj
problem with:  ANSYS_UNIGE_17_10.obj
ANSYS_UNIGE_27.mtl
ANSYS_UNIGE_27.obj
problem with:  ANSYS_UNIGE_27.obj
ANSYS_UNIGE_28_269.mtl
ANSYS_UNIGE_28_269.obj
problem with:  ANSYS_UNIGE_28_269.obj
ANSYS_UNIGE_30_612.mtl
ANSYS_UNIGE_30_612.obj
problem with:  ANSYS_UNIGE_30_612.obj
ANSYS_UNIGE_30_614.mtl
ANSYS_UNIGE_30_614.obj
problem with:  ANSYS_UNIGE_30_614.obj
ANSYS_UNIGE_33_628.mtl
ANSYS_UNIGE_33_628.obj
problem with:  ANSYS_UNIGE_33_628.obj
ANSYS_UNIGE_33_631.mtl
ANSYS_UNIGE_33_631.obj
problem with:  ANSYS_UNIGE_33_631.obj
ANSYS_UNIGE_34.mtl
ANSYS_UNIGE_34.obj
problem with:  ANSYS_UNIGE_34.obj
ANSYS_UNIGE_35.mtl
ANSYS_UNIGE_35.obj
problem with:  ANSYS_UNIGE_35.obj
C0001.mtl
C0001.obj
C0002.mtl
C0002.obj
C0003.mtl
C0003.obj
C0005.mtl
C0005.obj
C0006.mtl
C0006.obj
C0007.mtl
C0007.obj
C0008.mtl
C0008.obj

Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_10600\2996669240.py", line 12, in <module>
    grafo = calcularMatriz(fileObj, "../modelsNormalizado/radius_arrays/" + file.split(".")[0] + "-network_radius.npy")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 87, in calcularMatriz
    grafo, arbolVertices = calcularGrafoYArbol( fileObj, fileRadios )
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 29, in calcularGrafoYArbol
    return G, KDTree( verticesCrudos )
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\miniconda3\envs\vm\Lib\site-packages\scipy\spatial\_kdtree.py", line 360, in __init__
    super().__init__(data, leafsize, compact_nodes, copy_data,
  File "_ckdtree.

create binary tree radio

In [10]:
t_list = os.listdir('../modelsNormalizado/TreesNumpy/')
gfolder = os.listdir('../modelsNormalizado/grafos')   
print("Grafo files: ", gfolder)
print("TreesNumpy files: ", t_list)
files = os.listdir('../modelsNormalizado/vesselsOBJ/')
for file in files:
    
    if file.split(".")[0] + '-grafo.gpickle' in gfolder and file.split(".")[1] == 'obj' and (file.split(".")[0] + '.npy') not in t_list:

        try:
            grafo = pickle.load(open('../modelsNormalizado/grafos/' + file.split(".")[0] + '-grafo.gpickle',  'rb'))
            grafo = grafo.to_undirected()
            nc = 0
            #controlo si tiene ciclos
            if len(nx.cycle_basis(grafo))>0:
                nc = 1
            

            nb = 0
            #for nodo in grafo.nodes:
            for nodo in grafo.nodes:
                if len(grafo.edges(nodo))>3:
                    #print("binario", len(grafo.edges(nodo)))
                    binarizar(grafo)
                    #nb = 1
                    break
            
            if nb ==0 and nc == 0: 
                aRecorrer = []
                numeroNodoInicial = 1
                distancias = nx.floyd_warshall( grafo )

                parMaximo = (-1, -1)
                maxima = -1
            
                for nodoInicial in distancias.keys():
                    for nodoFinal in distancias[nodoInicial]:
                        if distancias[nodoInicial][nodoFinal] > maxima:
                            maxima = distancias[nodoInicial] [nodoFinal]
                            parMaximo = (nodoInicial, nodoFinal)

                for nodo in grafo.nodes:
                    if distancias[parMaximo[0]][nodo] == int( maxima / 2):
                        numeroNodoInicial = nodo
                        if len(grafo.edges(numeroNodoInicial))>2:
                            numeroNodoInicial = list(grafo.edges(numeroNodoInicial))[0][1]

                        break
            
        
                rad = list(grafo.nodes[numeroNodoInicial]['radio'])
                nodoRaiz = modelo.Node( numeroNodoInicial, radius =  rad )

                for vecino in grafo.neighbors( numeroNodoInicial ):
                    if vecino != numeroNodoInicial:
                        aRecorrer.append( (vecino, numeroNodoInicial,nodoRaiz ) )
                
                while len(aRecorrer) != 0:
                    nodoAAgregar, numeroNodoPadre,nodoPadre = aRecorrer.pop(0)
                    radius = list(grafo.nodes[nodoAAgregar]['radio'])
                    nodoActual = modelo.Node( nodoAAgregar, radius =  radius)
                    nodoPadre.agregarHijo( nodoActual )
                    for vecino in grafo.neighbors( nodoAAgregar ):
                        if vecino != numeroNodoPadre:
                            aRecorrer.append( (vecino, nodoAAgregar,nodoActual) )

    
                serial = nodoRaiz.serialize(nodoRaiz)
        
                f = []
                traversefeaturesSerializado(nodoRaiz, f, k=4)
            
                try:
                    array = np.array(f)
                    ##GUARDO
                    np.save("../modelsNormalizado/TreesNumpy/" + file.split(".")[0], array)
                    #print("flat", array)
                    print(f"calculated {file}")
                    print("number of nodes: ", array.shape)
                except Exception: 
                    for row in a:
                        if len(row) != 39:
                            breakpoint()
                    traceback.print_exc()
                    print(f"error with file{file}")

            else:
                print("error with file: ", file, nb, nc)
        except:
            print("error with file: ", file, nb, nc)

Grafo files:  ['C0001-grafo.gpickle', 'C0002-grafo.gpickle', 'C0003-grafo.gpickle', 'C0005-grafo.gpickle', 'C0006-grafo.gpickle', 'C0007-grafo.gpickle', 'C0008-grafo.gpickle', 'C0009-grafo.gpickle', 'C0010-grafo.gpickle', 'C0011-grafo.gpickle', 'C0012-grafo.gpickle', 'C0013-grafo.gpickle', 'C0014-grafo.gpickle', 'C0015-grafo.gpickle', 'C0016-grafo.gpickle', 'C0017-grafo.gpickle', 'C0018-grafo.gpickle', 'C0019-grafo.gpickle', 'C0020-grafo.gpickle', 'C0021-grafo.gpickle', 'C0023-grafo.gpickle', 'C0024-grafo.gpickle', 'C0025-grafo.gpickle', 'C0026-grafo.gpickle', 'C0027-grafo.gpickle', 'C0028a-grafo.gpickle', 'C0028b-grafo.gpickle', 'C0029-grafo.gpickle', 'C0030-grafo.gpickle', 'C0031-grafo.gpickle', 'C0032-grafo.gpickle', 'C0033-grafo.gpickle', 'C0034-grafo.gpickle', 'C0035-grafo.gpickle', 'C0036-grafo.gpickle', 'C0037-grafo.gpickle', 'C0038-grafo.gpickle', 'C0039-grafo.gpickle', 'C0040-grafo.gpickle', 'C0041-grafo.gpickle', 'C0042-grafo.gpickle', 'C0043-grafo.gpickle', 'C0044-grafo.gpic

In [11]:
t_list = os.listdir('../modelsNormalizado/Trees')
objfolder = os.listdir('../modelsNormalizado/VesselsOBJ')
for file in objfolder:
    if file.split(".")[0] + '-grafo.gpickle' in gfolder and file.split(".")[1] == 'obj' and (file.split(".")[0] + '_tree.dat') not in t_list:
        
        grafo = pickle.load(open('../modelsNormalizado/grafos/' + file.split(".")[0] + '-grafo.gpickle',  'rb'))
        grafo = grafo.to_undirected()

        nc = 0
        #controlo si tiene ciclos
        if len(nx.cycle_basis(grafo))>0:
            #print("ciclos", len(nx.cycle_basis(grafo)))
            nc = 1
        
        nb = 0
        #controlo si es binario (no hay nodos con mas de 3 edges)
        for nodo in grafo.nodes:
            if len(grafo.edges(nodo))>3:
                #print("binario", len(grafo.edges(nodo)))
                binarizar(grafo)
                #nb = 1
                break
        
        if nb ==0 and nc == 0: 
            try:
                serial = grafo2arbol(grafo)
                #write serialized string to file
                print("calculated file: ", file)
                file = open("../modelsNormalizado/Trees/" + file.split(".")[0] +"_tree.dat", "w")
                file.write(serial)
                file.close()
            except:
                print("error with file: ", file)
                traceback.print_exc()
        else:
            print("Not binary or with cycles: ", file)

Not binary or with cycles:  p095_EhMHDx4NAB8BEAgBFxMMFwAW_LICA.obj
error with file:  p321_BA4ACBEGGx8cFw8OHAgaCgcR_BA.obj
Not binary or with cycles:  p465_Ag4BBAINHBINChUKAAQAFgcX.obj
Not binary or with cycles:  p512_FQwfAB4dFw0fDBgCHwAAEBAU.obj


Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_10600\788567669.py", line 26, in <module>
    serial = grafo2arbol(grafo)
             ^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\splines.py", line 399, in grafo2arbol
    nodoPadre.agregarHijo( nodoActual )
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\Arbol.py", line 23, in agregarHijo
    raise ValueError ("solo arbol binario ")
ValueError: solo arbol binario 


creo grafo con splines

In [13]:
gfolder = sorted(os.listdir('../modelsNormalizado/grafosSplines'))
files = sorted(os.listdir('../modelsNormalizado/VesselsOBJ'))

l_error = []
for file in files:
   
    if file.split(".")[1] is not None:
        if file.split(".")[1] == "obj":
            try:
                #fileObj = open("resample_Eps01/centerlinesOBJ/" +file.split(".")[0] +"-network.obj")
                fileObj = open("../modelsNormalizado/centerlinesResampledOBJ/"+file.split(".")[0]+"-network.obj")
                #fileObj = open("centerlines/ArteryObjAN1-2-network.obj")
                
            except Exception:
                print("1 problem with: ", file)
                l_error.append(file)
                #traceback.print_exc()
            if file.split(".")[0] + '-grafo.gpickle' not in gfolder:
            #if True:
                try: 
                    grafo = calcularMatrizSplines(fileObj, "../modelsNormalizado/splines/coeficientes/" + file.split(".")[0] + ".pkl")
                    print("calculating: ", file)
                    with open("../modelsNormalizado/grafosSplines/" + file.split(".")[0] + '-grafo.gpickle', 'wb') as f:
                        #print(grafo.nodes('radio'))
                        pickle.dump(grafo, f, pickle.HIGHEST_PROTOCOL)
                except Exception:
                    print("2 problem with: ", file)
                    l_error.append(file)
                    traceback.print_exc()

gfolder = os.listdir('../modelsNormalizado/grafosSplines')  
#print("problem with files: ", l_error)

2 problem with:  ANSYS_UNIGE_09.obj
2 problem with:  ANSYS_UNIGE_16.obj
2 problem with:  ANSYS_UNIGE_17_10.obj
2 problem with:  ANSYS_UNIGE_27.obj
2 problem with:  ANSYS_UNIGE_28_269.obj
2 problem with:  ANSYS_UNIGE_30_612.obj
2 problem with:  ANSYS_UNIGE_30_614.obj
2 problem with:  ANSYS_UNIGE_33_628.obj
2 problem with:  ANSYS_UNIGE_33_631.obj
2 problem with:  ANSYS_UNIGE_34.obj
2 problem with:  ANSYS_UNIGE_35.obj
2 problem with:  SNF00000309.obj
2 problem with:  SNF00000426_03.obj
2 problem with:  SNF00000529.obj
2 problem with:  USFD_0001.obj
2 problem with:  USFD_0002.obj
2 problem with:  USFD_0004.obj
2 problem with:  USFD_0005.obj
2 problem with:  USFD_0006.obj
2 problem with:  USFD_0007.obj
2 problem with:  USFD_0008.obj
2 problem with:  USFD_0009.obj
2 problem with:  USFD_0011.obj
2 problem with:  USFD_0012.obj
2 problem with:  USFD_0013.obj
2 problem with:  USFD_0014.obj
2 problem with:  USFD_0016.obj
2 problem with:  USFD_0019.obj
2 problem with:  USFD_0020.obj
2 problem with

Traceback (most recent call last):
  File "C:\Users\lab03\AppData\Local\Temp\ipykernel_10600\955583907.py", line 21, in <module>
    grafo = calcularMatrizSplines(fileObj, "../modelsNormalizado/splines/coeficientes/" + file.split(".")[0] + ".pkl")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 103, in calcularMatrizSplines
    grafo, arbolVertices = calcularGrafoYArbolSplines( fileObj, fileRadios )
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\Documents\VesselGPTClean\Aneux\Preprocessing\parseObj.py", line 62, in calcularGrafoYArbolSplines
    return G, KDTree( verticesCrudos )
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lab03\miniconda3\envs\vm\Lib\site-packages\scipy\spatial\_kdtree.py", line 360, in __init__
    super().__init__(data, leafsize, compact_nodes, co

creo arbol con splines

In [ ]:
t_list = os.listdir('../modelsNormalizado/TreesSplines/')


for file in files:
   
    if file.split(".")[0] + '-grafo.gpickle' in gfolder and file.split(".")[1] == 'obj' and (file.split(".")[0] + '.npy') not in t_list:

        try:
            grafo = pickle.load(open('../modelsNormalizado/grafosSplines/' + file.split(".")[0] + '-grafo.gpickle',  'rb'))
            grafo = grafo.to_undirected()
            nc = 0
            #controlo si tiene ciclos
            if len(nx.cycle_basis(grafo))>0:
                nc = 1
            

            nb = 0
            #for nodo in grafo.nodes:
            for nodo in grafo.nodes:
                if len(grafo.edges(nodo))>3:
                    #print("binario", len(grafo.edges(nodo)))
                    binarizar(grafo)
                    #nb = 1
                    break
            
            if nb ==0 and nc == 0: 
                aRecorrer = []
                numeroNodoInicial = 1
                distancias = nx.floyd_warshall( grafo )

                parMaximo = (-1, -1)
                maxima = -1
            
                for nodoInicial in distancias.keys():
                    for nodoFinal in distancias[nodoInicial]:
                        if distancias[nodoInicial][nodoFinal] > maxima:
                            maxima = distancias[nodoInicial] [nodoFinal]
                            parMaximo = (nodoInicial, nodoFinal)

                for nodo in grafo.nodes:
                    if distancias[parMaximo[0]][nodo] == int( maxima / 2):
                        numeroNodoInicial = nodo
                        if len(grafo.edges(numeroNodoInicial))>2:
                            numeroNodoInicial = list(grafo.edges(numeroNodoInicial))[0][1]

                        break
            
        
                rad = list(grafo.nodes[numeroNodoInicial]['radio'])
                rad = limpiarRadiosSplines(rad)
                nodoRaiz = modelo.Node( numeroNodoInicial, radius =  rad )

                for vecino in grafo.neighbors( numeroNodoInicial ):
                    if vecino != numeroNodoInicial:
                        aRecorrer.append( (vecino, numeroNodoInicial,nodoRaiz ) )
                
                while len(aRecorrer) != 0:
                    nodoAAgregar, numeroNodoPadre,nodoPadre = aRecorrer.pop(0)
                    radius = list(grafo.nodes[nodoAAgregar]['radio'])
                    radius = limpiarRadiosSplines(radius)
                    nodoActual = modelo.Node( nodoAAgregar, radius =  radius)
                    nodoPadre.agregarHijo( nodoActual )
                    for vecino in grafo.neighbors( nodoAAgregar ):
                        if vecino != numeroNodoPadre:
                            aRecorrer.append( (vecino, nodoAAgregar,nodoActual) )

    
                serial = nodoRaiz.serialize(nodoRaiz)
        
                f = []
                traversefeaturesSerializado(nodoRaiz, f, k=39)
                print("number of nodes: ", len(f))
               
                
                array = np.array(f)
                ##GUARDO
                np.save("../modelsNormalizado/TreesSplines/" + file.split(".")[0], array)
                #print("flat", array)
                print(f"calculated {file}")
                
            else:
                print("error with file: ", file, nb, nc)
        except: 
            print("error with file: ", file, nb, nc)


number of nodes:  97
calculated C0001.obj
number of nodes:  125
calculated C0002.obj
number of nodes:  107
calculated C0003.obj
number of nodes:  81
calculated C0005.obj
number of nodes:  83
calculated C0006.obj
number of nodes:  185
calculated C0007.obj
number of nodes:  105
calculated C0008.obj
number of nodes:  93
calculated C0009.obj
number of nodes:  97
calculated C0010.obj
number of nodes:  113
calculated C0011.obj
number of nodes:  95
calculated C0012.obj
number of nodes:  97
calculated C0013.obj
number of nodes:  103
calculated C0014.obj
number of nodes:  115
calculated C0015.obj
number of nodes:  65
calculated C0016.obj
number of nodes:  117
calculated C0017.obj
number of nodes:  103
calculated C0018.obj
number of nodes:  117
calculated C0019.obj
number of nodes:  83
calculated C0020.obj
number of nodes:  89
calculated C0021.obj
number of nodes:  97
calculated C0023.obj
number of nodes:  85
calculated C0024.obj
number of nodes:  123
calculated C0025.obj
number of nodes:  69
ca